In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

# Configuración del WebDriver (en este caso, Chrome) para modo headless
options = Options()
options.headless = True  # Ejecutar el navegador en segundo plano

# Inicializar el WebDriver con ChromeDriver Manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL base de la página
base_url = 'https://www.carters.cl/category/cat14730004/Ropa-de-bebe-nino?page='

# Ruta del archivo Excel
excel_path = r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\CARTER_1_.xlsx'

# Variable para contar los productos
contador = 0
pagina = 1  # Empezamos desde la página 1

# Función para escribir los datos en el archivo Excel
def escribir_en_excel(datos_producto):
    # Crear un DataFrame con los datos del producto
    df_producto = pd.DataFrame([datos_producto])
    
    # Escribir los datos en el archivo Excel sin sobrescribir lo que ya existe
    try:
        with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            df_producto.to_excel(writer, index=False, sheet_name='Hoja1', header=False, startrow=writer.sheets['Hoja1'].max_row)
    except Exception as e:
        print(f"Error al guardar los datos en Excel: {e}")

# Función para extraer los productos de una página
def extraer_productos_de_pagina(url):
    global contador
    driver.get(url)
    
    # Esperar a que la página cargue completamente
    print(f"Accediendo a la página: {url}")
    time.sleep(5)  # Ajusta el tiempo de espera si es necesario
    
    # Establecer tiempo de espera para encontrar los elementos
    wait = WebDriverWait(driver, 30)  # Espera máxima de 30 segundos
    
    # Esperar a que el contenedor de productos esté visible
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid^="pod-"]')))  # Reemplaza con el selector adecuado si es necesario

    # Bucle para recorrer productos
    productos_encontrados = False
    try:
        # Encontrar todos los productos en la página
        productos = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid^="pod-"]')
        
        # Iterar sobre los productos encontrados
        for producto in productos:
            # Extraer el nombre del producto
            nombre_producto = producto.find_element(By.CSS_SELECTOR, 'p.chakra-text.css-jdl19f').text

            # Extraer la URL de la imagen
            imagen = producto.find_element(By.CSS_SELECTOR, 'img[data-testid="image"]')
            url_imagen = imagen.get_attribute('src')

            # Extraer el SKU de la URL de la imagen (antes de "/public")
            sku_match = re.search(r'(\d+)(?=/public)', url_imagen)
            if sku_match:
                sku = sku_match.group(1)
            else:
                sku = 'SKU no encontrado'

            # Extraer la URL de la página del producto
            enlace_producto = producto.find_element(By.CSS_SELECTOR, 'a.chakra-link.css-8tnj88')
            url_producto = enlace_producto.get_attribute('href')
            url_completa_producto = f'https://www.carters.cl{url_producto}'

            # Extraer el precio final del producto
            try:
                precio_final = producto.find_element(By.CSS_SELECTOR, 'div.Prices__PriceWithOffers-sc-1i3vp4r-2.eiJYdh p.chakra-text.css-6ekcoa').text.strip()
            except:
                precio_final = ""  # Si no se encuentra el precio final, dejamos el valor vacío

            # Extraer el precio original del producto
            try:
                precio_original = producto.find_element(By.CSS_SELECTOR, 'p.chakra-text.css-18qvutq').text.strip()
            except:
                precio_original = ""  # Si no se encuentra el precio original, dejamos el valor vacío

            # Almacenar los datos del producto en un diccionario
            producto_info = {
                'Nombre del producto': nombre_producto,
                'SKU': sku,
                'URL de la imagen': url_imagen,
                'URL del producto': url_completa_producto,
                'Precio final': precio_final,
                'Precio original': precio_original
            }

            # Escribir los datos del producto en el archivo Excel
            escribir_en_excel(producto_info)

            # Mostrar los datos del producto extraído
            print(f"Producto #{contador + 1}:")
            print(f"Nombre: {nombre_producto}")
            print(f"SKU: {sku}")
            print(f"Precio final: {precio_final}")
            print(f"Precio original: {precio_original}")
            print(f"URL de la imagen: {url_imagen}")
            print(f"URL del producto: {url_completa_producto}")
            print("-" * 40)  # Línea separadora entre productos

            # Incrementar el contador para el siguiente producto
            contador += 1
            productos_encontrados = True

        # Si no se encuentran productos
        if not productos_encontrados:
            print("No se encontraron productos en esta página.")
    except Exception as e:
        print(f"Error al extraer los productos: {e}")

    # Esperar antes de cargar la siguiente página
    print(f"Esperando antes de cargar la siguiente página...")
    time.sleep(3)  # Ajusta el tiempo de espera si es necesario

    return productos_encontrados

# Bucle para recorrer todas las páginas
while True:
    # Generar la URL de la página actual
    url_pagina = f'{base_url}{pagina}'
    
    # Extraer productos de la página actual
    productos_encontrados = extraer_productos_de_pagina(url_pagina)
    
    if not productos_encontrados:
        print(f"No se encontraron productos en la página {pagina}. Terminando el script.")
        break  # Si no se encontraron productos en la página, terminamos el bucle
    
    # Incrementar la página
    pagina += 1

# Cerrar el navegador
driver.quit()


Accediendo a la página: https://www.carters.cl/category/cat14730004/Ropa-de-bebe-nino?page=1
Error al guardar los datos en Excel: [Errno 2] No such file or directory: 'C:\\Users\\constanza.perez\\OneDrive - Colgram\\Escritorio\\CARTER_1_.xlsx'
Producto #1:
Nombre: Pack de Bodys Algodón 5 unidades Bebé Niño
SKU: 1
Precio final: $ 29.990
Precio original: 
URL de la imagen: https://www.carters.cl/cdn-cgi/imagedelivery/4fYuQyy-r8_rpBpcY7lH_A/falabellaCL/17176029_1/public
URL del producto: https://www.carters.clhttps://www.carters.cl/products/17176028/Pack-de-Bodys-Algodon-5-unidades-Bebe-Nino-Carter's/17176029
----------------------------------------
Error al guardar los datos en Excel: [Errno 2] No such file or directory: 'C:\\Users\\constanza.perez\\OneDrive - Colgram\\Escritorio\\CARTER_1_.xlsx'
Producto #2:
Nombre: Bodys Básicos Algodón Manga Larga Unisex Bebé
SKU: 01
Precio final: $ 24.990
Precio original: 
URL de la imagen: https://www.carters.cl/cdn-cgi/imagedelivery/4fYuQyy-r8_rpBp

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x009B74A3+25091]
	(No symbol) [0x0093DC04]
	(No symbol) [0x0081B373]
	(No symbol) [0x0085F4DC]
	(No symbol) [0x0085F65B]
	(No symbol) [0x0089D8E2]
	(No symbol) [0x00881F54]
	(No symbol) [0x0089B49E]
	(No symbol) [0x00881CA6]
	(No symbol) [0x008531D5]
	(No symbol) [0x0085435D]
	GetHandleVerifier [0x00CB07C3+3142947]
	GetHandleVerifier [0x00CC1A2B+3213195]
	GetHandleVerifier [0x00CBC412+3191154]
	GetHandleVerifier [0x00A58720+685184]
	(No symbol) [0x00946E1D]
	(No symbol) [0x00943E18]
	(No symbol) [0x00943FB6]
	(No symbol) [0x009366F0]
	BaseThreadInitThunk [0x75B05D49+25]
	RtlInitializeExceptionChain [0x77A9CEBB+107]
	RtlGetAppContainerNamedObjectPath [0x77A9CE41+561]
